In [1]:
%matplotlib inline
# ---- settings ----
import json
import matplotlib.pyplot as plt
from os.path import isfile
import sys
from time import time
import cv2
import numpy as np
from os.path import join, isdir
from mpl_toolkits.mplot3d import Axes3D
sys.path.insert(0,'../')
sys.path.append('./../../easy_multi_person_pose_estimation')
from mvpose import pose
from poseestimation import model
pe = model.PoseEstimator()
import mvpose.data.transform as tfm

Settings = json.load(open('../settings.txt'))

root = join(Settings['data_root'], 'pak')

from pak.datasets.UMPM import UMPM
user = Settings['UMPM']['username']
pwd = Settings['UMPM']['password']


X, Y, Calib = tfm.get_from_umpm(root, 'p2_free_1', user, pwd)

# create video
start_frame = 0
skip = 25
total_frames = 100



/home/julian/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


UMPM


In [ ]:
from mvpose.pose import estimate
from mvpose.plot.skeleton import generate3d
import mvpose.plot.limbs as pltlimbs
from mvpose.data.default_limbs import  DEFAULT_LIMB_SEQ

def plot_onto(ax, humans, frame):
    ax.set_xlim([-1500, 2500])
    ax.set_ylim([-1500, 2500])
    ax.set_zlim([0, 2000])
    ax.set_xlabel('x');  ax.set_ylabel('y')
    COLOR = ['orange', 'teal', 'cornflowerblue', '#B0BF1A', '#C9FFE5', '#008000']
    COLOR_GT = ['red', 'blue']

    for i,pid in enumerate(humans.keys()):
        human = humans[pid]
        for k, p in enumerate(human['joints']):
            if p is not None:
                x,y,z,w = p
                ax.scatter(x,y,z,color=COLOR[i], alpha=1, s=w*20)
                #ax.text(x,y,z, str(k))

        generate3d(ax, human['joints'], DEFAULT_LIMB_SEQ, COLOR[i], alpha=0.4)

    for x,y,z,pid in Y[frame]:
        ax.scatter([x],[y],[z], color=COLOR_GT[int(pid)], alpha=0.4, s=10)

        
# ~~~~~~~~~~~~~~~~~~~~~~~~

with_gpu = False
idx = 0
for f in range(start_frame, (start_frame + total_frames) * skip, skip):
    
    print('handling frame ', f)
    
    Im = np.array([X[0][f], X[1][f], X[2][f], X[3][f]])
    
    _start = time()
    if with_gpu:
        heatmaps, pafs = pe.predict_pafs_and_heatmaps(I)
    else:
        if isfile('/tmp/heatmaps.npy') and isfile('/tmp/pafs.npy'):
            heatmaps = np.load('/tmp/heatmaps.npy')
            pafs = np.load('/tmp/pafs.npy')
        else:
            hm_file = '/tmp/heatmaps' + str(f) + '.npy'
            paf_file = '/tmp/pafs' + str(f) + '.npy'

            if isfile(hm_file) and isfile(paf_file):
                heatmaps = np.load(hm_file)
                pafs = np.load(paf_file)
            else:
                heatmaps = []; pafs = []
                for im in Im:
                    print('...')
                    hm, paf = pe.predict_pafs_and_heatmaps(im)
                    heatmaps.append(np.squeeze(hm))
                    pafs.append(np.squeeze(paf))
                heatmaps = np.array(heatmaps)
                pafs = np.array(pafs)
                np.save(hm_file, heatmaps)
                np.save(paf_file, pafs)
    
    humans, candidates = estimate(Calib, heatmaps, pafs,
                     threshold_nbr_multiview_modes=1, 
                     threshold_drop_person=7,
                     debug_info=True)
    
    fname = 'vid/frame%04d.png' % (idx,)
    idx += 1
    
    
    _end = time(); print('elapsed:', _end - _start)
    
    
    fig = plt.figure(figsize=(20, 20))
    fig.add_subplot(4,4,1).imshow(Im[0])
    fig.add_subplot(4,4,2).imshow(Im[1])
    fig.add_subplot(4,4,3).imshow(Im[2])
    fig.add_subplot(4,4,4).imshow(Im[3])
    
    peaks = candidates.peaks2d
    limbs = candidates.limbs2d
    
    for cid in range(4):
        ax = fig.add_subplot(4,4,5+cid)
        pltlimbs.plot(ax, Im[cid], peaks[cid], limbs[cid]); ax.axis('off')
        
    ax = fig.add_subplot(223, projection='3d'); plot_onto(ax, humans, f)
    ax = fig.add_subplot(224, projection='3d'); ax.view_init(45, 0); 
    plot_onto(ax, humans, f)

    

    plt.savefig(fname, format='png')
    
    

handling frame  0
elapsed: 2.6425626277923584
handling frame  25
elapsed: 2.6348214149475098
handling frame  50
elapsed: 2.6171023845672607
handling frame  75
elapsed: 2.648505687713623
handling frame  100
elapsed: 2.5360896587371826
handling frame  125
elapsed: 2.637240171432495
handling frame  150
elapsed: 2.5571298599243164
handling frame  175
elapsed: 2.585606575012207
handling frame  200
elapsed: 2.6552085876464844
handling frame  225
elapsed: 2.5833113193511963
handling frame  250
elapsed: 2.586287260055542
handling frame  275
elapsed: 2.59157133102417
handling frame  300
elapsed: 2.604743003845215
handling frame  325
elapsed: 2.7654223442077637
handling frame  350
elapsed: 2.6064319610595703
handling frame  375
elapsed: 2.5650951862335205
handling frame  400
elapsed: 2.663254737854004
handling frame  425
elapsed: 2.60732364654541
handling frame  450
elapsed: 2.6149914264678955
handling frame  475
elapsed: 2.608668804168701
handling frame  500
elapsed: 2.6855456829071045


/home/julian/anaconda3/lib/python3.6/site-packages/matplotlib/pyplot.py:537: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


handling frame  525
elapsed: 2.689429759979248
handling frame  550
elapsed: 2.685859441757202
handling frame  575
elapsed: 2.664546489715576
handling frame  600
elapsed: 2.583693027496338
handling frame  625
elapsed: 2.5895144939422607
handling frame  650
elapsed: 2.6527631282806396
handling frame  675
elapsed: 2.5929338932037354
handling frame  700
elapsed: 2.591987133026123
handling frame  725
elapsed: 2.611387252807617
handling frame  750
elapsed: 2.64579176902771
handling frame  775
elapsed: 2.724656105041504
handling frame  800
elapsed: 2.702150344848633
handling frame  825
elapsed: 2.718146800994873
handling frame  850
elapsed: 2.625023126602173
handling frame  875
elapsed: 2.7974417209625244
handling frame  900
elapsed: 3.1592791080474854
handling frame  925
elapsed: 2.713066577911377
handling frame  950
elapsed: 2.9093542098999023
handling frame  975
elapsed: 2.589881420135498
handling frame  1000
elapsed: 2.5677950382232666
handling frame  1025
elapsed: 2.6242730617523193
hand